# Energium-Celform Test 2020.9.17

## Connect Indy7 and Pickit

In [1]:
from indy_utils import indydcp_client as client
from indy_utils.indy_program_maker import JsonProgramComponent
from vision_clients import pickit_client as p_client

from time import sleep
import json
from threading import Lock
import threading

In [2]:
#Set
robot_ip = "192.168.1.14"
name = "NRMK-Indy7" #Set robot name
bind_ip = "192.168.1.20" # PC IP
pickit_addr = "192.168.1.24" # Pickit Controller IP
pickit_port = 5002

# Connect to robot
indy = client.IndyDCPClient(robot_ip, name)
indy.connect()

# Robot status (putty: pkill -9 TaskMan)
status = indy.get_robot_status()
print(status)

# Pickit connect
pickit = p_client.PickitClient(bind_ip, pickit_addr, pickit_port)
pickit.connect()

Connect: Server IP (192.168.1.14)
{'ready': 1, 'emergency': 0, 'collision': 0, 'error': 0, 'busy': 0, 'movedone': 0, 'home': 0, 'zero': 0, 'resetting': 0, 'teaching': 0, 'direct_teaching': 0}
Connect: Bind IP (192.168.1.20), Server IP (192.168.1.24)


True

In [3]:
#  Pickit should receive periodic robot flange pose updates from the client.
PERIOD_STOP = False
TERMINATE_THREADS = False

def periodic_send():
    while not PERIOD_STOP:
        sleep(0.1)
        pickit.periodic(list(indy.get_task_pos()))  
        if TERMINATE_THREADS:
            print('periodic_send finished')
            break

# Create each thread for 'periodic update'
th2 = threading.Thread(target=periodic_send)
th2.start()

## Config Pickit setup and product files

In [ ]:
setup_id = 11
product_id = 31

status = pickit.request_config(indy, setup_id, product_id)

In [ ]:
pickit.check_mode(indy)

## Detection

In [6]:
obj_pos, obj_remaning, status = pickit.get_object_pos(indy)
pickit.check_object_found(status)

size res data 64


False

In [7]:
obj_pos

array([0., 0., 0., 0., 0., 0.])

## Teaching pick point

In [ ]:
## Move Indy to the pick point
pickit.set_tool_offset(indy, obj_pos)

## Gripper on

In [ ]:
indy.set_do(8, 1)

## Gripper off

In [30]:
indy.set_do(8, 0)

# Teach waypoints

In [ ]:
file_name='teaching/teach_config.json'

# Add teaching points - task_move position

# Move robot to proper waypoints of every step in the whole process.
# Execute this cell per every waypoints after moving robot.

# You MUST choose joint_move or task_move.

indy.update_teaching_data(file_name, 'entry', indy.get_joint_pos())

## Move direct to the pick point

In [24]:
vel=3
pick_height=0.05  # You should set the velocity of robot and pick height (meters) from pick position.

# Load teaching data
with open('teaching/teach_config.json') as json_file:
    teach_config = json.load(json_file)
    
# Load teaching data
with open('teaching/tool_config.json') as json_file:
    tool_config = json.load(json_file)
    
t_place_2 = teach_config['t_place']
    
for i in range(5):
    # Obtatin pick position from Pickit based on current robot position
    pick_pos, pick_above, objects_remaining, status = pickit.get_pickit_pos(indy, pick_height, tool_config['tool_offset_1'])
    t_place_2[2] += 0.003
    # Wait next program to be finished after first iterat|ion
    # In this case, wait for 'Place' program to be done after getting pick position above
#     indy.wait_for_program_finish()

    # Before program starts, check there are pickable oject and pickit is running
    # Also, check the workspace was set

    if pickit.check_object_found(status) and pickit.check_mode(indy) and indy.set_workspace(pick_pos):
        print('Detect! Move to pick!')
        print(pick_pos)
        print(pick_above)

        # Pick
        prog = JsonProgramComponent(policy=3, resume_time=2)  # Make Json program
        prog.add_digital_out(8, 0)  # Release gripper
        prog.add_move_home()

        # Move to pick position by task move
        prog.add_joint_move_to(teach_config['entry'], vel=vel)
        prog.add_task_move_to(list(pick_above), vel=vel, blend=0.1)
        prog.add_task_move_to(list(pick_pos), vel=vel, blend=0.1)
        prog.add_digital_out(8, 1)  # Release gripper

        prog.add_wait(0.3)
        prog.add_task_move_to(list(pick_above), vel=vel, blend=0.1)
        prog.add_task_move_to(teach_config['pre_place'], vel=vel)
        prog.add_task_move_to(t_place_2, vel=vel)
        prog.add_digital_out(8, 0)
        prog.add_wait(0.3)

        prog.add_task_move_to(teach_config['pre_place'], vel=vel)
        prog.add_move_home()

        indy.set_and_start_json_program(prog.program_done())
        indy.wait_for_program_finish()
    else:
        print('No object is detected')

size res data 64
size res data 64
Detect! Move to pick!
[ 5.54650128e-01  6.28075423e-03  5.20209447e-02 -1.79200700e+02
 -1.40799994e-01 -1.75964600e+02]
[ 5.54723620e-01  6.98516890e-03  1.02015935e-01 -1.79200700e+02
 -1.40799994e-01 -1.75964600e+02]
Program Finished:  True
size res data 64
size res data 64
Detect! Move to pick!
[ 5.56053042e-01  6.89564599e-03  4.50163409e-02 -1.78879200e+02
 -5.37400014e-01 -1.74373200e+02]
[ 5.56423783e-01  7.91492593e-03  9.50045735e-02 -1.78879200e+02
 -5.37400014e-01 -1.74373200e+02]
Program Finished:  True
size res data 64
size res data 64
Detect! Move to pick!
[ 5.54227889e-01  5.22761047e-03  3.92227210e-02 -1.79441400e+02
  7.22000021e-02 -1.74949700e+02]
[ 5.54122210e-01  5.70763461e-03  8.92203003e-02 -1.79441400e+02
  7.22000021e-02 -1.74949700e+02]
Program Finished:  True
size res data 64
size res data 64
Detect! Move to pick!
[ 5.53310573e-01  4.43278346e-03  3.29163820e-02 -1.78760600e+02
 -2.13000006e-02 -1.72060000e+02]
[ 5.5317956

In [5]:
tcp = [0.0, 0.0, 0.035, 0.0, 0.0, 0.0]
indy.set_default_tcp(tcp)
indy.get_default_tcp()

[0.0, 0.0, 0.035, 0.0, 0.0, 0.0]

In [8]:
vel=3
pick_height=0.05  # You should set the velocity of robot and pick height (meters) from pick position.

# Load teaching data
with open('teaching/teach_config.json') as json_file:
    teach_config = json.load(json_file)
    
# Load teaching data
with open('teaching/tool_config.json') as json_file:
    tool_config = json.load(json_file)

# 1
setup_id = 12
product_id = 31
status = pickit.request_config(indy, setup_id, product_id)
sleep(1)

pick_pos, pick_above, objects_remaining, status = pickit.get_pickit_pos(indy, pick_height, tool_config['tool_offset_1'])

if pickit.check_object_found(status) and pickit.check_mode(indy) and indy.set_workspace(pick_pos):
    print('Detect! Move to pick!')
    print(pick_pos)
    print(pick_above)
        
    # Pick
    prog = JsonProgramComponent(policy=3, resume_time=2)  # Make Json program
    prog.add_digital_out(8, 0)  # Release gripper
    prog.add_move_home()

    # Move to pick position by task move
    prog.add_joint_move_to(teach_config['entry'], vel=vel)
    prog.add_task_move_to(list(pick_above), vel=vel, blend=0.1)
    prog.add_task_move_to(list(pick_pos), vel=vel, blend=0.1)
    prog.add_digital_out(8, 1)  # Release gripper

    prog.add_wait(0.3)
    prog.add_task_move_to(list(pick_above), vel=vel, blend=0.1)
    prog.add_task_move_to(teach_config['pre_place'], vel=vel)
    prog.add_task_move_to(teach_config['t_place_1'], vel=vel)
    prog.add_digital_out(8, 0)
    prog.add_wait(0.3)

    prog.add_task_move_to(teach_config['pre_place'], vel=vel)
    prog.add_move_home()

    indy.set_and_start_json_program(prog.program_done())

else:
    print('No object is detected')

  

  
# 2
setup_id = 13
product_id = 32
status = pickit.request_config(indy, setup_id, product_id)
indy.wait_for_program_finish()

pick_pos, pick_above, objects_remaining, status = pickit.get_pickit_pos(indy, pick_height, tool_config['tool_offset_1'])

if pickit.check_object_found(status) and pickit.check_mode(indy) and indy.set_workspace(pick_pos):
    print('Detect! Move to pick!')
    print(pick_pos)
    print(pick_above)
    
    # Pick
    prog = JsonProgramComponent(policy=3, resume_time=2)  # Make Json program
    prog.add_digital_out(8, 0)  # Release gripper
    prog.add_move_home()

    # Move to pick position by task move
    prog.add_joint_move_to(teach_config['entry'], vel=vel)
    prog.add_task_move_to(list(pick_above), vel=vel, blend=0.1)
    prog.add_task_move_to(list(pick_pos), vel=vel, blend=0.1)
    prog.add_digital_out(8, 1)  # Release gripper

    prog.add_wait(0.3)
    prog.add_task_move_to(list(pick_above), vel=vel, blend=0.1)
    prog.add_task_move_to(teach_config['pre_place'], vel=vel)
    prog.add_task_move_to(teach_config['t_place_2'], vel=vel)
    prog.add_digital_out(8, 0)
    prog.add_wait(0.3)

    prog.add_task_move_to(teach_config['pre_place'], vel=vel)
    prog.add_move_home()

    indy.set_and_start_json_program(prog.program_done())
else:
    print('No object is detected')

    
# 3
setup_id = 14
product_id = 33
status = pickit.request_config(indy, setup_id, product_id)


indy.wait_for_program_finish()
#sleep(15)
pick_pos, pick_above, objects_remaining, status = pickit.get_pickit_pos(indy, pick_height, tool_config['tool_offset_1'])


if pickit.check_object_found(status) and pickit.check_mode(indy) and indy.set_workspace(pick_pos):
    print('Detect! Move to pick!')
    print(pick_pos)
    print(pick_above)
    

    # Pick
    prog = JsonProgramComponent(policy=3, resume_time=2)  # Make Json program
    prog.add_digital_out(8, 0)  # Release gripper
    prog.add_move_home()

    # Move to pick position by task move
    prog.add_joint_move_to(teach_config['entry'], vel=vel)
    prog.add_task_move_to(list(pick_above), vel=vel, blend=0.1)
    prog.add_task_move_to(list(pick_pos), vel=vel, blend=0.1)
    prog.add_digital_out(8, 1)  # Release gripper

    prog.add_wait(0.3)
    prog.add_task_move_to(list(pick_above), vel=vel, blend=0.1)
    prog.add_task_move_to(teach_config['pre_place'], vel=vel)
    prog.add_task_move_to(teach_config['t_place_3'], vel=vel)
    prog.add_digital_out(8, 0)
    prog.add_wait(0.3)

    prog.add_task_move_to(teach_config['pre_place'], vel=vel)
    prog.add_move_home()

    indy.set_and_start_json_program(prog.program_done())
    indy.wait_for_program_finish()
else:
    print('No object is detected')
    

size res data 64
size res data 64
size res data 64
Detect! Move to pick!
[ 5.53584397e-01 -9.30991992e-02  3.55237275e-02 -1.79683800e+02
 -1.53699997e-01  8.76460001e+01]
[ 5.53854585e-01 -9.32445452e-02  8.55227858e-02 -1.79683800e+02
 -1.53699997e-01  8.76460001e+01]
size res data 64
Program Finished:  True
size res data 64
size res data 64
Detect! Move to pick!
[ 5.78660190e-01  1.66092422e-02  2.86235511e-02  1.79755500e+02
 -3.12900000e-01  9.17673000e+01]
[ 5.78455329e-01  1.63297392e-02  7.86223561e-02  1.79755500e+02
 -3.12900000e-01  9.17673000e+01]
size res data 64
Program Finished:  True
size res data 64
size res data 64
Detect! Move to pick!
[ 5.87036967e-01  1.53208762e-01  2.71240957e-02 -1.79769800e+02
  1.16999996e-02  9.07528000e+01]
[ 5.87237716e-01  1.53221607e-01  7.71236867e-02 -1.79769800e+02
  1.16999996e-02  9.07528000e+01]
Program Finished:  True


In [16]:
# Emergency stop example program

indy.stop_current_program()
indy.stop_motion()

Header check fail (cmdId): Request 17, Response 9999
ErrorCode 19: No proper program state


## Teaching place point

In [58]:
file_name='teaching/teach_config.json'

# Add teaching points - task_move position

# Move robot to proper waypoints of every step in the whole process.
# Execute this cell per every waypoints after moving robot.

# You MUST choose joint_move or task_move.

indy.update_teaching_data(file_name, 'entry', indy.get_joint_pos())

{'t_pick': [0.5571096188007364,
  0.16528509628718466,
  0.46108681852340266,
  0.39710131561674084,
  179.49770306118722,
  18.233124554065693],
 't_prepick': [0.5397469873327341,
  0.02542943635159567,
  0.3144541516890686,
  179.16340053842453,
  0.44584854024171416,
  178.9294426598851],
 'j_detect': [4.539805010330878e-05,
  -14.999969734634254,
  -90.00013619415624,
  5.438776299505309e-05,
  -74.99996374149627,
  -5.438776299505309e-05],
 'j_pre_entry': [-21.309890116543244,
  -29.958400253423978,
  -84.92104774664816,
  -0.19492574257427028,
  -66.16309439782304,
  69.65152551632576],
 't_entry': [0.3957395589871438,
  0.10188531496211345,
  0.4071773508999492,
  2.536804027126256,
  -175.9546966034077,
  0.01695406757047043],
 'j_home': [-68.86702608471528,
  -25.192240565278002,
  -103.45902371998356,
  -1.1459501663057685,
  -55.78357054455813,
  49.69301129332011],
 't_place1': [0.4318104197024063,
  0.2786032223113445,
  0.2015476029427117,
  0.013037626531245454,
  -179.9

In [10]:
# Add teaching points - joint_move position

# Move robot to proper waypoints of every step in the whole process.
# Execute this cell per every waypoints after moving robot.

# You MUST choose joint_move or task_move.

indy.update_teaching_data(file_name, 'j_entry', indy.get_joint_pos())

{'t_pick': [0.5571096188007364,
  0.16528509628718466,
  0.46108681852340266,
  0.39710131561674084,
  179.49770306118722,
  18.233124554065693],
 't_prepick': [0.5397469873327341,
  0.02542943635159567,
  0.3144541516890686,
  179.16340053842453,
  0.44584854024171416,
  178.9294426598851],
 'j_detect': [4.539805010330878e-05,
  -14.999969734634254,
  -90.00013619415624,
  5.438776299505309e-05,
  -74.99996374149627,
  -5.438776299505309e-05],
 'j_pre_entry': [-21.309890116543244,
  -29.958400253423978,
  -84.92104774664816,
  -0.19492574257427028,
  -66.16309439782304,
  69.65152551632576],
 't_entry': [0.4948164610855849,
  -0.23933260574684476,
  0.27983213796130685,
  -179.1423527365628,
  -0.1469067785351539,
  89.16629367121625],
 'j_home': [-68.86702608471528,
  -25.192240565278002,
  -103.45902371998356,
  -1.1459501663057685,
  -55.78357054455813,
  49.69301129332011],
 't_place1': [0.4318104197024063,
  0.2786032223113445,
  0.2015476029427117,
  0.013037626531245454,
  -179

In [7]:
indy.get_default_tcp()

[0.0, 0.0, 0.035, 0.0, 0.0, 0.0]

In [9]:
# Add teaching points - task_move position

# Move robot to proper waypoints of every step in the whole process.
# Execute this cell per every waypoints after moving robot.

# You MUST choose joint_move or task_move.
file_name='teaching/teach_config.json'
indy.update_teaching_data(file_name, 't_place_3', indy.get_task_pos())

{'t_pick': [0.5571096188007364,
  0.16528509628718466,
  0.46108681852340266,
  0.39710131561674084,
  179.49770306118722,
  18.233124554065693],
 't_prepick': [0.5397469873327341,
  0.02542943635159567,
  0.3144541516890686,
  179.16340053842453,
  0.44584854024171416,
  178.9294426598851],
 'j_detect': [4.539805010330878e-05,
  -14.999969734634254,
  -90.00013619415624,
  5.438776299505309e-05,
  -74.99996374149627,
  -5.438776299505309e-05],
 'j_pre_entry': [-21.309890116543244,
  -29.958400253423978,
  -84.92104774664816,
  -0.19492574257427028,
  -66.16309439782304,
  69.65152551632576],
 't_entry': [0.3957395589871438,
  0.10188531496211345,
  0.4071773508999492,
  2.536804027126256,
  -175.9546966034077,
  0.01695406757047043],
 'j_home': [-68.86702608471528,
  -25.192240565278002,
  -103.45902371998356,
  -1.1459501663057685,
  -55.78357054455813,
  49.69301129332011],
 't_place1': [0.4318104197024063,
  0.2786032223113445,
  0.2015476029427117,
  0.013037626531245454,
  -179.9

In [5]:
# Remove teaching points

# Execute this cell when you need to remove specific waypoint with the name of waypoints.

indy.del_teaching_pos('j_test')

{'tool_offset': [-0.000799, -0.00384, 0.0931, -179.5, -1.819, 94.0],
 'j_home': [33.78, 10.195, -107.94, 0.0005984, -82.25, 10.89],
 'j_pre_entry': [75.0, -21.81, -116.94, 2.875, -43.12, -0.001414],
 't_pre_entry': [0.2715, 0.3245, 0.2207, -1.8955, -178.1, 72.94],
 't_entry': [0.359, 0.456, 0.1498, -1.673, -178.0, 68.25],
 't_place1': [0.03001, 0.2903, 0.2208, -1.88, -178.1, 72.9],
 't_place2': [-0.201, 0.2905, 0.266, -1.94, -178.1, 72.9]}

In [12]:
# Remove teaching points

# Execute this cell when you need to remove specific waypoint with the name of waypoints.

indy.del_teaching_pos('t_test')

{'tool_offset': [-0.000799, -0.00384, 0.0931, -179.5, -1.819, 94.0],
 'j_home': [33.78, 10.195, -107.94, 0.0005984, -82.25, 10.89],
 'j_pre_entry': [75.0, -21.81, -116.94, 2.875, -43.12, -0.001414],
 't_pre_entry': [0.2715, 0.3245, 0.2207, -1.8955, -178.1, 72.94],
 't_entry': [0.359, 0.456, 0.1498, -1.673, -178.0, 68.25],
 't_place1': [0.03001, 0.2903, 0.2208, -1.88, -178.1, 72.9],
 't_place2': [-0.201, 0.2905, 0.266, -1.94, -178.1, 72.9]}

# Load Config data

In [26]:
# Load teaching data
with open('teaching/teach_config.json') as json_file:
    teach_config = json.load(json_file)

In [77]:
# Load teaching data
with open('teaching/tool_config.json') as json_file:
    tool_config = json.load(json_file)

In [48]:
tool_config['tool_offset_1']

[0.0018260478973388672,
 0.001470952294766903,
 0.03609694540500641,
 -179.38510101622774,
 -1.085901347999017,
 0.9499213678999416]

# Pickit example program 

In [22]:
#  Pickit should receive periodic robot flange pose updates from the client.
PERIOD_STOP = False
TERMINATE_THREADS = False

def periodic_send():
    while not PERIOD_STOP:
        sleep(0.1)
        pickit.periodic(list(indy.get_task_pos()))  
        if TERMINATE_THREADS:
            print('periodic_send finished')
            break

# Create each thread for 'periodic update'
th2 = threading.Thread(target=periodic_send)
th2.start()

In [8]:
pick_pos, pick_above, objects_remaining, status = pickit.get_pickit_pos(indy, 0.03, tool_config['tool_offset1'])

print(pick_pos)
print(pick_above)

size res data 64
[ 4.31451589e-01  4.75360185e-01  1.48531735e-01  1.78296456e+02
 -1.75407031e+00 -1.17005104e+02]
[   0.43216401    0.47662964    0.17849639  178.29645593   -1.75407031
 -117.00510366]


In [28]:
vel=1
pick_height=0.05  # You should set the velocity of robot and pick height (meters) from pick position.

# Obtatin pick position from Pickit based on current robot position
pick_pos, pick_above, objects_remaining, status = pickit.get_pickit_pos(indy, pick_height, tool_config['tool_offset2'])

# Wait next program to be finished after first iteration
# In this case, wait for 'Place' program to be done after getting pick position above
indy.wait_for_program_finish()

# Before program starts, check there are pickable oject and pickit is running
# Also, check the workspace was set
if pickit.check_object_found(status) and pickit.check_mode(indy) and indy.set_workspace(pick_pos):
    print('Detect! Move to pick!')
    print(pick_pos)
    print(pick_above)

    # Pick
    prog = JsonProgramComponent(policy=3, resume_time=2)  # Make Json program
    prog.add_endtool_do(type=0, value=0)  # Release gripper
    prog.add_joint_move_to(teach_config['j_detect_home'], vel=vel, blend=20)  # Move to pre-entry point by joint move
    prog.add_joint_move_to(teach_config['j_entry'], vel=vel)  # Move to entry point by task move
#     prog.add_task_move_to(teach_config['t_entry'], vel=vel)  # Move to entry point by task move

    # Check pick position from Pickit is lower than the height of entry point 
    if pick_above[2] <= teach_config['t_entry'][2]:
        # Move to pick position by task move
        prog.add_task_move_to(list(pick_above), vel=vel, blend=0.1)
        prog.add_task_move_to(list(pick_pos), vel=vel, blend=0.1)
        prog.add_endtool_do(type=0, value=1)  # Release gripper
        

    # Move backward to entry and pre entry points
    prog.add_wait(1)
    prog.add_joint_move_to(teach_config['j_entry'], vel=vel)
    prog.add_joint_move_to(teach_config['j_detect_home'], vel=vel)

    indy.set_and_start_json_program(prog.program_done())
    indy.wait_for_program_finish()  # Wait 'Pick' program to be finished

    # Place
    prog = JsonProgramComponent(policy=3, resume_time=2)                
    prog.add_task_move_to(teach_config['t_place'], vel=vel, blend=0.1)
    prog.add_endtool_do(type=0, value=0)  # Release gripper
    prog.add_wait(0.3)
    prog.add_joint_move_to(teach_config['j_detect_home'], vel=vel)

    indy.set_and_start_json_program(prog.program_done())               

else:
    print('No object is detected')

size res data 64
Program Finished:  True
size res data 64
Detect! Move to pick!
[4.01893169e-01 4.75239754e-01 1.32928416e-01 1.68031702e+02
 2.22670806e+00 6.74419818e+01]
[  0.39396808   0.47963294   0.18210049 168.03170181   2.22670806
  67.44198184]
Program Finished:  True


In [28]:
vel=1
pick_height=0.05  # You should set the velocity of robot and pick height (meters) from pick position.

# Obtatin pick position from Pickit based on current robot position
pick_pos, pick_above, objects_remaining, status = pickit.get_pickit_pos(indy, pick_height, tool_config['tool_offset2'])

# Wait next program to be finished after first iteration
# In this case, wait for 'Place' program to be done after getting pick position above
indy.wait_for_program_finish()

# Before program starts, check there are pickable oject and pickit is running
# Also, check the workspace was set
if pickit.check_object_found(status) and pickit.check_mode(indy) and indy.set_workspace(pick_pos):
    print('Detect! Move to pick!')
    print(pick_pos)
    print(pick_above)

    # Pick
    prog = JsonProgramComponent(policy=3, resume_time=2)  # Make Json program
    prog.add_endtool_do(type=0, value=0)  # Release gripper
    
#     prog.add_digital_out(idx=8, val=0)
#     dis = indy.get_di()
#     is_gripped = dis[7]
    
    prog.add_joint_move_to(teach_config['j_detect_home'], vel=vel, blend=20)  # Move to pre-entry point by joint move
    prog.add_joint_move_to(teach_config['j_entry'], vel=vel)  # Move to entry point by task move
#     prog.add_task_move_to(teach_config['t_entry'], vel=vel)  # Move to entry point by task move

    # Check pick position from Pickit is lower than the height of entry point 
    if pick_above[2] <= teach_config['t_entry'][2]:
        # Move to pick position by task move
        prog.add_task_move_to(list(pick_above), vel=vel, blend=0.1)
        prog.add_task_move_to(list(pick_pos), vel=vel, blend=0.1)
        prog.add_endtool_do(type=0, value=1)  # Release gripper
        

    # Move backward to entry and pre entry points
    prog.add_wait(1)
    prog.add_joint_move_to(teach_config['j_entry'], vel=vel)
    prog.add_joint_move_to(teach_config['j_detect_home'], vel=vel)

    indy.set_and_start_json_program(prog.program_done())
    indy.wait_for_program_finish()  # Wait 'Pick' program to be finished

    # Place
    prog = JsonProgramComponent(policy=3, resume_time=2)                
    prog.add_task_move_to(teach_config['t_place'], vel=vel, blend=0.1)
    prog.add_endtool_do(type=0, value=0)  # Release gripper
    prog.add_wait(0.3)
    prog.add_joint_move_to(teach_config['j_detect_home'], vel=vel)

    indy.set_and_start_json_program(prog.program_done())               

else:
    print('No object is detected')

size res data 64
Program Finished:  True
size res data 64
Detect! Move to pick!
[4.01893169e-01 4.75239754e-01 1.32928416e-01 1.68031702e+02
 2.22670806e+00 6.74419818e+01]
[  0.39396808   0.47963294   0.18210049 168.03170181   2.22670806
  67.44198184]
Program Finished:  True


In [19]:
# Emergency stop example program

indy.stop_motion()
indy.stop_current_program()

TERMINATE_THREADS = True

Header check fail (cmdId): Request 17, Response 9999
ErrorCode 19: No proper program state
